In [2]:
!pip install nlp-id

     |████████████████████████████████| 7.5MB 48kB/s 
     |████████████████████████████████| 7.0MB 46.5MB/s 
     |████████████████████████████████| 1.5MB 41.6MB/s 
  Created wheel for nlp-id: filename=nlp_id-0.1.9.8-cp36-none-any.whl size=7723018 sha256=a61fcc5cff7a5920f3b26e7286a203b8f1c9697087c9662ff0d6324b05f1e983
  Stored in directory: /root/.cache/pip/wheels/87/04/4e/25d7de62878a993a6642138ef4c9b5ea2cdc78f12cf8624c4d
  Created wheel for nltk: filename=nltk-3.4.5-cp36-none-any.whl size=1449907 sha256=cd1f8070b0bb6a66d2ba74e01001caffa6da7919541bc931bc24e117be32408f
  Stored in directory: /root/.cache/pip/wheels/96/86/f6/68ab24c23f207c0077381a5e3904b2815136b879538a24b483
  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=22153ed5d352fbc41f3fc072f1e2883a5208c2a54052b810fd43785d7b4c8d4d
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built nlp-id nltk wget
  Found existing installation

In [4]:
import nltk
nltk.download('punkt')
from nltk.tag import CRFTagger
from nlp_id.postag import PosTag
import numpy as np
postagger = PosTag() 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


No model removed


/usr/local/lib/python3.6/dist-packages/nlp_id/postag.py:39: UserWarning: Downloading model ..
  warnings.warn("Downloading model ..")


In [5]:
import pandas as pd
baca = pd.read_csv('data/Data_Preprocessed.csv')
stop = baca['Stopword'].tolist()

In [8]:
from nltk import word_tokenize
from nlp_id.lemmatizer import Lemmatizer 
lemmatizer = Lemmatizer() 
from nlp_id.tokenizer import PhraseTokenizer 
tokenizer = PhraseTokenizer() 

In [10]:
def cleantag(ls):
    pos2 = []
    tag = ["IN", "SC", "CC", "PR", "JJ"]
    for i in ls:
        if i[1] not in tag:
            pos2.append(i)
    return pos2

In [12]:
def append_multiple_lines(file_name, lines_to_append):
    # Open the file in append & read mode ('a+')
    with open(file_name, "a+") as file_object:
        appendEOL = False
        # Move read cursor to the start of file.
        file_object.seek(0)
        # Check if file is not empty
        data = file_object.read(100)
        if len(data) > 0:
            appendEOL = True
        # Iterate over each string in the list
        for line in lines_to_append:
            # If file is not empty then append '\n' before first line for
            # other lines always append '\n' before appending line
            if appendEOL == True:
                file_object.write("\n")
            else:
                appendEOL = True
            # Append element at the end of file
            file_object.write(line)

NER

In [ ]:
def nltk_parse_clause(sentence):
    grammar = r"""
    STRN: {<NUM><NN|NUM><VB>*}
            }<NUM><NN.*|NUM><VB>{
    INGRD: {(<FW|NNP>*<STRN>)|(<NNP|FW>*<NN><STRN>)}
            }<STRN>{
    MED: {<NNP>+?<NUM><NNP|NUM|NN>}
    MED: {<NNP|NN|FW>*<VB><STRN>}
          }<VB><STRN>{
    MED: {<NN.*|FW>*<VB><INGRD>}
            }<VB><INGRD>{
    MED: {<NN.*>*<VB><NUM>}
          }<VB><NUM>{
    INGRD: {<MED><VB><STRN><NN.*|FW>*}
          }<MED><VB><STRN>{
    """
    cp = nltk.RegexpParser(grammar)
    parsed_sentence = cp.parse(sentence)
    return parsed_sentence

In [2]:
chunked = []
for i in range(len(stop)):
    t = stop[i].replace('%', ' %').replace('unit-internasional', 'unit')
    temp = cleantag(postagger.get_pos_tag(t))
    result = nltk_parse_clause(temp)
    chunked.append(result)

In [ ]:
from nltk.chunk import conlltags2tree, tree2conlltags
BIOres = []
for i in chunked:
    iob_tagged = tree2conlltags(i)
    BIOres.append(iob_tagged)

NOUN PHRASE

In [ ]:
def noun_phrase(sentence):
    grammar = r"""
    NP: {<NN>*}
    NP: {<FW>*}
    NP: {<NNP>*}
    VB: {<VB>*}
    """
    cp = nltk.RegexpParser(grammar)
    parsed_sentence = cp.parse(sentence)
    return parsed_sentence

In [3]:
chunkedNP = []
for i in range(len(stop)):
    t = stop[i].replace('%', ' %').replace('unit-internasional', 'unit')
    temp = cleantag(postagger.get_pos_tag(t))
    result = noun_phrase(temp)
    chunkedNP.append(result)

In [ ]:
BIOnp = []
for i in chunkedNP:
    iob_tagged = tree2conlltags(i)
    BIOnp.append(iob_tagged)

NOW APPEND

In [ ]:
dataset = []
for h in range(len(BIOres)):
    for i, b in zip(BIOres[h], BIOnp[h]):
        dataset.append(i[0] + " " +i[1] + " " + b[2] + " " + i[2])
    dataset.append("")

In [ ]:
append_multiple_lines('Dataset_NER.txt', tr)